# ADVANCED TEXT MINING
- 본 자료는 텍스트 마이닝을 활용한 연구 및 강의를 위한 목적으로 제작되었습니다.
- 본 자료를 강의 목적으로 활용하고자 하시는 경우 꼭 아래 메일주소로 연락주세요.
- 본 자료에 대한 허가되지 않은 배포를 금지합니다.
- 강의, 저작권, 출판, 특허, 공동저자에 관련해서는 문의 바랍니다.
- **Contact : ADMIN(admin@teanaps.com)**

---

## WEEK 11-1. 키워드 추출 (Keyword Extraction)
- Python으로 문서에서 키워드 추출하는 방법에 대해 다룹니다.

---

In [ ]:
# 텍스트 분석을 위한 TEANAPS 패키지를 설치합니다.
# TEANAPS는 Google Colaboratory/Linux 환경에 최적화되어 있습니다.
# Windows 환경에서 일부 기능에 제한이 있을 수 있습니다.

In [ ]:
# TEANAPS (https://github.com/fingeredman/teanaps)
#!git clone https://github.com/fingeredman/teanaps.git

In [ ]:
#!ls

In [ ]:
# TEANAPS 설치를 진행합니다.
# 설치 전 반드시 상단 메뉴에서 [런타임 > 런타임 초기화]를 클릭한 후 진행해주세요.
#!python "teanaps/teanaps_setup.py"

### 1. 학습 데이터 준비하기

---

#### 1.1. TEANAPS 라이브러리 불러오기

---

In [ ]:
from teanaps.nlp import MorphologicalAnalyzer
from teanaps.nlp import Processing
from teanaps.handler import FileHandler
from teanaps.nlp import NamedEntityRecognizer
from teanaps.nlp import SyntaxAnalyzer

from sklearn.model_selection import train_test_split

ma = MorphologicalAnalyzer()
ma.set_tagger("mecab")
ma.set_tagger("mecab-ko")
#ner = NamedEntityRecognizer(model_path="/model")
ner = NamedEntityRecognizer()
sa = SyntaxAnalyzer()
tfidf = TfidfCalculator()
processing = Processing()
fh = FileHandler()

#### 1.2. 데이터 파일 불러오기

---

In [ ]:
import pandas as pd

df = pd.read_csv("data/kea_train_set.csv")
df

In [ ]:
df = df.dropna()
df

#### 1.3. 학습/평가 데이터 준비하기

---

In [ ]:
from sklearn.model_selection import train_test_split
x = df[['tfidf', "first_occurrence"]]
y = df["is_keyword"]
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, test_size=0.2)

### 2. 학습하기

---

#### 2.1. 학습: 랜덤 포레스트 (Random Forest)

---

In [ ]:
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier()
random_forest.fit(x_train, y_train)         # 학습
score = random_forest.score(x_test, y_test) # 평가
print("Accuracy:", score)

# 학습모델 저장
fh.save_data("data/rf_model", random_forest)

#### 2.2. 학습: 서포트 벡터 머신 (SVM)

---

In [ ]:
from sklearn.svm import SVC

svm_model = SVC(probability=True)
svm_model.fit(x_train, y_train)         # 학습
score = svm_model.score(x_test, y_test) # 평가
print('Accuracy :', score)

# 학습모델 저장
fh.save_data("data/svm_model", svm_model)

#### 2.3. 학습: KNN

---

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn_model = KNeighborsClassifier(n_neighbors=20)
knn_model.fit(x_train, y_train)         # 학습
score = knn_model.score(x_test, y_test) # 평가
print('Accuracy :', score)

# 학습모델 저장
fh.save_data("data/knn_model", knn_model)

#### 2.4. 학습: LDA

---

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

lda_model = LinearDiscriminantAnalysis()
lda_model.fit(x_train, y_train)         # 학습
score = lda_model.score(x_test, y_test) # 평가
print('Accuracy :', score)

# 학습모델 저장
fh.save_data("data/lda_model", lda_model)

### 3. 학습모델 활용하기

---

In [ ]:
tfidf_dict = fh.load_data("data/tfidf_dict")

In [ ]:
query = "높은 주파수의 신호 사용시에 그 신호의 주파수를 정확하고 일정하게 제어하는 집적회로(IC)"
pos_result = ma.parse(query_lower)
ner_result = ner.parse(query_lower)
sa_result = sa.parse(pos_result, ner_result)

candidate_list = []
first_occurrence = 0
for word, pos, _, _ in sa_result:
    first_occurrence += 1
    if pos in ["NNG", "NNP"]:
        if word in tfidf_dict.keys():
            candidate_list.append([word, tfidf_dict[word], first_occurrence])
        else:
            candidate_list.append([word, 0, first_occurrence])

candidate_list

In [ ]:
model = fh.load_data("data/rf_model")

for candidate in candidate_list:
    input_data = [candidate[1], candidate[2]]
    candidate_prob_list = model.predict_proba([input_data]).tolist()[0]
    if candidate_prob_list[1] > 0.5:
        is_keyword = 1
    else:
        is_keyword = 0
    print(candidate[0], is_keyword, input_data, candidate_prob_list)